<h1> Created by: Vinayak Gupta, Vaishali Jain </h1>

In [1]:
import nltk
import numpy as np
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\vinay\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [2]:
sentence = ['she', 'was', 'not']
n = 3
corpus = [
    w.lower() for w in
    nltk.corpus.gutenberg.words('austen-sense.txt')
    ]

In [3]:
#Use to create a dictionary of dictionary where the outer dictionary contains te last n-1 tokens and 
#the inner dictionary is just a frequency of the nth token

def preprocess_probdist(corpus,n,prob_dist):
    if(n==1):
        return prob_dist
    for idx in range(n-1,len(corpus)):
        tup= tuple(corpus[idx-n+1:idx])
        current_word= corpus[idx]
        if tup not in prob_dist:
            prob_dist[tup] = {current_word:1}
        else:
            if current_word in prob_dist[tup]:
                prob_dist[tup][current_word]+=1
            else:
                prob_dist[tup][current_word]=1
    return prob_dist

In [4]:
#Used to find the best word to return from the inner dictionary deterministically as well as non-deterministically
def helper(last_n_gram,prob_dist,deterministic):
    max_val=0
    if(deterministic):
        for i in prob_dist[last_n_gram]:
            if prob_dist[last_n_gram][i]> max_val:
                max_val = prob_dist[last_n_gram][i]
                best_word= i
            elif(prob_dist[last_n_gram][i]==max_val):
                if(i>best_word):
                    best_word=i        
    else:           
        keys = prob_dist[last_n_gram].keys()
        values = prob_dist[last_n_gram].values()
        val_sum = sum(values)
        values = [float(i)/val_sum for i in values]
        best_word = np.random.choice(list(keys),p = values)
    return best_word

In [5]:
#Special case to create a dictionary when a unigram is being used in a non-deterministic way
def non_deterministic_unigram(corpus):
    d={}
    for i in corpus:
        if i in d:
            d[i]+=1
        else:
            d[i]=1
    keys = d.keys()
    values = d.values()
    values = [float(i)/len(corpus) for i in values]
    return keys,values

In [6]:
def finish_sentence(sentence, n, corpus, deterministic):
    end_words = ['.', '?', '!']
    prob_dist={}
    prob_dist=preprocess_probdist(corpus,n,prob_dist)
    #print(prob_dist)
    min_ngram_in_dict=n
    n_gram_reach_1=0
    while(sentence[-1][0] not in end_words and len(sentence)<15):
        last_n_gram= tuple(sentence[-(n-1):])
        gram_length=n
        appended=0
        while(gram_length>1):
            if gram_length< min_ngram_in_dict:
                prob_dist=preprocess_probdist(corpus,gram_length,prob_dist)
                min_ngram_in_dict-=1
            last_n_gram = last_n_gram[(n-gram_length):]
            #print(last_n_gram)
            if last_n_gram in prob_dist:
                best_word=helper(last_n_gram,prob_dist,deterministic)
                sentence.append(best_word)
                appended=1
                break
            gram_length-=1
        if(appended==0):
            if(deterministic):
                if (n_gram_reach_1==0):
                    max_word=max(set(corpus), key = corpus.count)
                #print(max_word)
                    n_gram_reach_1=1
            else:
                if(n_gram_reach_1==0):
                    unigram_keys, unigram_values = non_deterministic_unigram(corpus)
                max_word = np.random.choice(list(unigram_keys),p = unigram_values)
            sentence.append(max_word) 
    #print(prob_dist[(',')])
    return sentence

**Non-deterministic case**

In [7]:
sentence = ['she', 'was', 'not']
sentence_1 = finish_sentence(sentence, n, corpus, False)
sentence_1

['she',
 'was',
 'not',
 'very',
 'material',
 ';',
 'and',
 'as',
 'it',
 'is',
 'very',
 'considerable',
 ';',
 'and',
 'secretly']

**Deterministic case**

In [8]:
sentence = ['she', 'was', 'not']
sentence_2 = finish_sentence(sentence, n, corpus, True)
sentence_2

['she',
 'was',
 'not',
 'in',
 'the',
 'world',
 ',',
 'and',
 'the',
 'two',
 'miss',
 'steeles',
 ',',
 'as',
 'she']